# word2vec implementation

In [1]:
import sys
import os
import re
from typing import Dict, List
import numpy as np
import tensorflow as tf

np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(linewidth=400) 

%load_ext line_profiler
%load_ext autoreload

In [2]:
from itertools import islice

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

# Overview
<img src="image/word2vec_cbow_mechanism.png" align="left"/>

# Constants

In [3]:
USE_PTB = True
DEBUG = False
VALIDATION = True

# Text utilities

In [4]:
%autoreload 2
import function.fileio as fileio

# Corpus

In [5]:
corpus = "To be, or not to be, that is the question that matters"
_file = "ptb.train.txt"
if USE_PTB:
    if not fileio.Function.is_file(f"~/.keras/datasets/{_file}"):
        path_to_ptb = tf.keras.utils.get_file(
            _file, 
            f'https://raw.githubusercontent.com/tomsercu/lstm/master/data/{_file}'
        )
    corpus = fileio.Function.read_file(path_to_ptb)

In [13]:
examples = corpus.split('\n')[:5]
for line in examples:
    print(line)

 aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memotec mlx nahb punts rake regatta rubens sim snack-food ssangyong swapo wachter 
 pierre <unk> N years old will join the board as a nonexecutive director nov. N 
 mr. <unk> is chairman of <unk> n.v. the dutch publishing group 
 rudolph <unk> N years old and former chairman of consolidated gold fields plc was named a nonexecutive director of this british industrial conglomerate 
 a form of asbestos once used to make kent cigarette filters has caused a high percentage of cancer deaths among a group of workers exposed to it more than N years ago researchers reported 


# Word indexing

In [6]:
%autoreload 2
from layer.preprocessing import (
    WordIndexing, 
    EventContext
)

## WordIndexing instance for the corpus

Adapt to the ```corpus``` and provides:
* word_to_index dictionary
* vocaburary of the corpus
* word occurrence probabilites

In [7]:
word_indexing = WordIndexing(
    name="word_indexing_on_ptb",
    corpus=corpus
)

In [8]:
print(f"WordIndexing.vocabulary[10]:\n{word_indexing.vocabulary[:10]}\n")
print(f"WordIndexing.word_to_index[10]:")
for item in take(10, word_indexing.word_to_index.items()):
    print(item)

print(f"\nWordIndexing.probabilities[10]:")
for item in take(10, word_indexing.word_to_index.items())[1:11]:
    print(f"{item[0]:15}: {word_indexing.probabilities[item[0]]}")


WordIndexing.vocabulary[10]:
['<nil>' '<unk>' 'lee' 'profits' 'consume' 'elevators' 'difficulty' 'bartlett' 'aim' 'yields']

WordIndexing.word_to_index[10]:
('<nil>', 0)
('<unk>', 1)
('lee', 2)
('profits', 3)
('consume', 4)
('elevators', 5)
('difficulty', 6)
('bartlett', 7)
('aim', 8)
('yields', 9)

WordIndexing.probabilities[10]:
<unk>          : 0.05056069360915978
lee            : 6.176895043322497e-05
profits        : 0.00016509155843061945
consume        : 7.86150278241045e-06
elevators      : 1.0107646434527721e-05
difficulty     : 2.8076795651465893e-05
bartlett       : 3.0322939303583165e-05
aim            : 2.2461436521172715e-05
yields         : 0.00013252247547491903


## Sentence to Sequence

In [12]:
word_indexing.function("\n")

AssertionError: 

In [10]:
# sentences = "\n".join(corpus.split('\n')[5:6])
sentences = """
the asbestos fiber <unk> is unusually <unk> once it enters the <unk> with even brief exposures to it causing symptoms that show up decades later researchers said
"""
sequences = word_indexing.function(sentences)
for pair in zip(sentences.strip().split(" "), sequences[0]):
    print(f"{pair[0]:15} : {pair[1]:5}")

AssertionError: 